<a href="https://colab.research.google.com/github/BrunoArroyo/Analise-Exploratoria-de-Logistica/blob/main/an_lise_explorat_ria_de_dados_logisticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística II

Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Manipulação;</li>
  <li>Visualização;</li>
  <li>Storytelling.</li>
</ol>


---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

 O projeto se baseia em dados fictícios (porém, semelhantes ao que a empresa lida) fornecidos pela Startup Loggi.

 O problema essencial é otimizar o planejamento de rotas dos veículos para realizar todas as entregas, respeitando as capacidades dos veículos e minimizando o custo total, que pode envolver tempo, distância percorrida, ou outros critérios de otimização.

 É um problema clássico em logística, onde o objetivo é alocar eficientemente os veículos para atender às demandas de entrega, considerando as restrições de capacidade e otimizando as rotas para reduzir custos operacionais.







## 2\. Pacotes e bibliotecas

In [1]:
# Instalando e importando todas as bibliotecas utilizadas no projeto
!pip3 install geopandas
!pip install tabula-py PyPDF2

import json

import tabula
import PyPDF2
from PyPDF2 import PdfReader
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.2 MB/s eta 0:00:00


## 3\. Exploração de dados

**3.1 Carregamento dos Dados**

 Para poder fazer a exploração dos dados vou precisar extrair do arquivo json e colocar em um dataframe pandas.


In [2]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

# Transformando o dicionário python em um DataFrame do pandas
deliveries_df = pd.DataFrame(data)

# Visualize o DataFrame deliveries_df e perceba um problema na coluna origin
deliveries_df.head()

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,"{'lng': -47.89366206897872, 'lat': -15.8051175...",180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


**3.2 Data Wrangling**

 Eis a questão: há dados nested nas colunas "origin", então precisamos utilizar o método "json_normalize" do pandas para separar essas informações em colunas de um novo DataFrame.


In [3]:
# Separando origin em duas colunas e armazenando em uma variável
hub_origin_df = pd.json_normalize(deliveries_df["origin"])

# Visualize como ela ficou
hub_origin_df.head()

,lng,lat
0,-48.054989,-15.838145
1,-48.054989,-15.838145
2,-48.054989,-15.838145
3,-47.893662,-15.805118
4,-48.054989,-15.838145


 Agora vamos juntar essas novas colunas com o nosso DataFrame principal.

In [4]:
# Usando a função merge posso definir deliveries_df a esquerda e hub_origin_df a direita
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

# Agora vou tirar a coluna origin do DataFrame, pois se torna desnecessária
deliveries_df = deliveries_df.drop("origin", axis=1)

# Renomeio as colunas para ficar mais organizado
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]

# Explicito a lat e lng que são referentes ao hub
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)
deliveries_df.head()

,name,region,hub_lng,hub_lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."
1,cvrp-2-df-73,df-2,-48.054989,-15.838145,180,"[{'id': 'bf3fc630b1c29601a4caf1bdd474b85', 'po..."
2,cvrp-2-df-20,df-2,-48.054989,-15.838145,180,"[{'id': 'b30f1145a2ba4e0b9ac0162b68d045c3', 'p..."
3,cvrp-1-df-71,df-1,-47.893662,-15.805118,180,"[{'id': 'be3ed547394196c12c7c27c89ac74ed6', 'p..."
4,cvrp-2-df-87,df-2,-48.054989,-15.838145,180,"[{'id': 'a6328fb4dc0654eb28a996a270b0f6e4', 'p..."


 Observe que a situação da coluna deliveries é ainda mais complexa do que com a origin que trabalhamos há pouco. Nela temos uma lista dentro de uma lista. Para podermos separar tudo corretamente iremos começar usando a função explode, ela irá criar uma linha para cada dicionário dentro da coluna.


In [5]:
# Aplicando o método "explode" à coluna "deliveries"
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
deliveries_exploded_df

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."
0,"{'id': '320c94b17aa685c939b3f3244c3099de', 'po..."
0,"{'id': '3663b42f4b8decb33059febaba46d5c8', 'po..."
0,"{'id': 'e11ab58363c38d6abc90d5fba87b7d7', 'poi..."
0,"{'id': '54cb45b7bbbd4e34e7150900f92d7f4b', 'po..."
...,...
198,"{'id': '21693bf442ac5890adbdf2648c12881a', 'po..."
198,"{'id': '7aaa35088b37b6e542c4cd69663a7ebf', 'po..."
198,"{'id': '60c00d5390da4f28167439cd9c566703', 'po..."
198,"{'id': '51f456963785e7381243ff7baf7efd06', 'po..."


In [6]:
# Criando um DataFrame diferente com uma coluna de cada informação e concatenando em um único DataFrame
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["id"])).rename(columns={"deliveries":"delivery_id"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)

deliveries_normalized_df

,delivery_size,delivery_id,delivery_lng,delivery_lat
0,9,313483a19d2f8d65cd5024c8d215cfbd,-48.116189,-15.848929
0,2,320c94b17aa685c939b3f3244c3099de,-48.118195,-15.850772
0,1,3663b42f4b8decb33059febaba46d5c8,-48.112483,-15.847871
0,2,e11ab58363c38d6abc90d5fba87b7d7,-48.118023,-15.846471
0,7,54cb45b7bbbd4e34e7150900f92d7f4b,-48.114898,-15.858055
...,...,...,...,...
198,8,21693bf442ac5890adbdf2648c12881a,-48.064269,-15.997694
198,4,7aaa35088b37b6e542c4cd69663a7ebf,-48.065176,-16.003597
198,9,60c00d5390da4f28167439cd9c566703,-48.065841,-16.003808
198,1,51f456963785e7381243ff7baf7efd06,-48.062327,-16.001568


In [7]:
# Excluindo a coluna "deliveries" original
deliveries_df = deliveries_df.drop("deliveries", axis=1)

# Mesclando o novo DataFrame deliveries_normalized_df com o DataFrame original deliveries_df, right join pelo index
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)

deliveries_df

,name,region,hub_lng,hub_lat,vehicle_capacity,delivery_size,delivery_id,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,9,313483a19d2f8d65cd5024c8d215cfbd,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,2,320c94b17aa685c939b3f3244c3099de,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,1,3663b42f4b8decb33059febaba46d5c8,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,2,e11ab58363c38d6abc90d5fba87b7d7,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,7,54cb45b7bbbd4e34e7150900f92d7f4b,-48.114898,-15.858055
...,...,...,...,...,...,...,...,...,...
636144,cvrp-2-df-62,df-2,-48.054989,-15.838145,180,8,21693bf442ac5890adbdf2648c12881a,-48.064269,-15.997694
636145,cvrp-2-df-62,df-2,-48.054989,-15.838145,180,4,7aaa35088b37b6e542c4cd69663a7ebf,-48.065176,-16.003597
636146,cvrp-2-df-62,df-2,-48.054989,-15.838145,180,9,60c00d5390da4f28167439cd9c566703,-48.065841,-16.003808
636147,cvrp-2-df-62,df-2,-48.054989,-15.838145,180,1,51f456963785e7381243ff7baf7efd06,-48.062327,-16.001568


**3.3 Análise Exploratória**

 Neste momento os nossos dados já foram tabulados corretamente e podemos seguir para a análise exploratória.

In [8]:
# Conferindo o tamanho lateral e horizontal da tabela em questão
deliveries_df.shape

(636149, 9)

In [9]:
# Reconhecendo a classificação de cada coluna
deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              636149 non-null  object 
 1   region            636149 non-null  object 
 2   hub_lng           636149 non-null  float64
 3   hub_lat           636149 non-null  float64
 4   vehicle_capacity  636149 non-null  int64  
 5   delivery_size     636149 non-null  int64  
 6   delivery_id       636149 non-null  object 
 7   delivery_lng      636149 non-null  float64
 8   delivery_lat      636149 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 43.7+ MB


In [10]:
# Interpretando onde temos valores nulos e suas possíveis implicações no projeto
deliveries_df.isna().sum()

# Neste caso não temos nulos, então não há necessidade de se preocupar

name                0
region              0
hub_lng             0
hub_lat             0
vehicle_capacity    0
delivery_size       0
delivery_id         0
delivery_lng        0
delivery_lat        0
dtype: int64

In [11]:
# Observando os valores distintos nas colunas.
deliveries_df.nunique()

name                   199
region                   3
hub_lng                  3
hub_lat                  3
vehicle_capacity         1
delivery_size           10
delivery_id         291566
delivery_lng        291566
delivery_lat        291566
dtype: int64

In [12]:
# Fazendo a separação dos dados categóricos dos numéricos
var_categoricas = []
var_numericos = []

for coluna in deliveries_df.columns:
  if deliveries_df[coluna].dtype != "object":
    var_numericos.append(coluna)
  else:
    var_categoricas.append(coluna)

print(f'numéricas: {var_numericos}')
print(f'categóricas: {var_categoricas}')

numéricas: ['hub_lng', 'hub_lat', 'vehicle_capacity', 'delivery_size', 'delivery_lng', 'delivery_lat']
categóricas: ['name', 'region', 'delivery_id']


In [13]:
# Vamos puxar um resumo estatístico dos dados numéricos para já ir ententendo melhor o que pode ser esperado dessa base de dados
deliveries_df[var_numericos].describe().T

,count,mean,std,min,25%,50%,75%,max
hub_lng,636149.0,-47.949902,0.091875,-48.054989,-48.054989,-47.893662,-47.893662,-47.802665
hub_lat,636149.0,-15.802359,0.053463,-15.838145,-15.838145,-15.805118,-15.805118,-15.657014
vehicle_capacity,636149.0,180.000000,0.000000,180.000000,180.000000,180.000000,180.000000,180.000000
delivery_size,636149.0,5.512111,2.874557,1.000000,3.000000,6.000000,8.000000,10.000000
delivery_lng,636149.0,-47.946087,0.112769,-48.280779,-48.035911,-47.928967,-47.883394,-47.310611
delivery_lat,636149.0,-15.809492,0.082462,-16.050028,-15.842795,-15.814033,-15.769516,-15.500355


Alguns Insights:

- A capacidade do veículo é constante em 180, indicando que todos os veículos têm a mesma capacidade de carga.
- As coordenadas do hub indicam uma localização geográfica estável para a distribuição.
- As coordenadas de entrega têm uma dispersão maior do que as coordenadas do hub, indicando uma distribuição mais ampla das entregas.

In [14]:
# Vamos puxar um resumo estatístico dos dados categóricos para entender o que pode ser interpretado
deliveries_df[var_categoricas].describe().T

,count,unique,top,freq
name,636149,199,cvrp-1-df-87,5636
region,636149,3,df-1,304708
delivery_id,636149,291566,61b87669243974d021c2b76fc5272045,12


Alguns Insights:
- O nome "cvrp-1-df-87" é um identificador comum e recorrente no conjunto de dados, sugerindo possivelmente um padrão ou categoria específica.
- A região "df-1" é a mais comum entre as regiões, representando a maioria dos registros.
- O ID de entrega "61b87669243974d021c2b76fc5272045" aparece repetidamente, indicando que várias entregas têm o mesmo ID.

## 4\. Manipulação

 Através de *Geocodificação Reversa* vamos descobrir os nomes dos locais por trás das coordenadas geográficas dos hubs e das entregas envolvidas no projeto.

 Iniciaremos separando as regiões do DataFrame principal e apagando as repetições para termos uma lista definida.

In [15]:
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]]
hub_df = hub_df.drop_duplicates().sort_values(by="region").reset_index(drop=True)
hub_df.head()

,region,hub_lng,hub_lat
0,df-0,-47.802665,-15.657014
1,df-1,-47.893662,-15.805118
2,df-2,-48.054989,-15.838145


 Através da ferramenta gratuita Nominatim, a geocodificação reversa é feita.

In [16]:
# definindo as variaveis separadamente
geolocator = Nominatim(user_agent="ebac_geocoder")
location = geolocator.reverse("-15.657013854445248, -47.802664728268745")
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

hub_df["coordinates"] = hub_df["hub_lat"].astype(str)  + ", " + hub_df["hub_lng"].astype(str)
hub_df["geodata"] = hub_df["coordinates"].apply(geocoder)
hub_df.head()

,region,hub_lng,hub_lat,coordinates,geodata
0,df-0,-47.802665,-15.657014,"-15.657013854445248, -47.802664728268745","(Rua 7, Quadra 2, Sobradinho, Região Geográfic..."
1,df-1,-47.893662,-15.805118,"-15.80511751066334, -47.89366206897872","(SQS 303, Asa Sul, Brasília, Plano Piloto, Reg..."
2,df-2,-48.054989,-15.838145,"-15.83814451122274, -48.05498915846707","(Armazém do Bolo, lote 4/8, CSB 4/5, Taguating..."


In [20]:
hub_geodata_df = pd.json_normalize(hub_df["geodata"].apply(lambda data: data.raw))
hub_geodata_df.head()

,place_id,licence,osm_type,osm_id,lat,lon,class,type,place_rank,importance,...,address.state,address.ISO3166-2-lvl4,address.region,address.postcode,address.country,address.country_code,address.neighbourhood,address.city,address.shop,address.house_number
0,46410411,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,240210480,-15.656916027876347,-47.80264463632131,highway,secondary,26,0.10001,...,Distrito Federal,BR-DF,Região Centro-Oeste,73015-202,Brasil,br,NaN,NaN,NaN,NaN
1,46401425,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,66353368,-15.805172757199022,-47.893723539491205,highway,residential,26,0.10001,...,Distrito Federal,BR-DF,Região Centro-Oeste,70342-010,Brasil,br,SQS 303,Brasília,NaN,NaN
2,43777060,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,6249717596,-15.8384371,-48.0552917,shop,pastry,30,0.00001,...,Distrito Federal,BR-DF,Região Centro-Oeste,72015-030,Brasil,br,NaN,Taguatinga,Armazém do Bolo,lote 4/8


In [21]:
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]
hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)
hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(), hub_geodata_df["hub_suburb"], hub_geodata_df["hub_city"])
hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)
hub_geodata_df.head()

<ipython-input-21-e7ffd1c368d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)
<ipython-input-21-e7ffd1c368d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
<ipython-input-21-e7ffd1c368d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,hub_suburb,hub_city
0,Sobradinho,Sobradinho
1,Asa Sul,Brasília
2,Taguatinga,Taguatinga


In [22]:
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "hub_suburb", "hub_city"]]
hub_df.head()

,region,hub_suburb,hub_city
0,df-0,Sobradinho,Sobradinho
1,df-1,Asa Sul,Brasília
2,df-2,Taguatinga,Taguatinga


In [23]:
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")
deliveries_df = deliveries_df[["name", "region", "hub_lng", "hub_lat", "hub_city", "hub_suburb", "vehicle_capacity", "delivery_size", "delivery_lng", "delivery_lat"]]
deliveries_df.head()

,name,region,hub_lng,hub_lat,hub_city,hub_suburb,vehicle_capacity,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,9,-48.116189,-15.848929
1,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,2,-48.118195,-15.850772
2,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,1,-48.112483,-15.847871
3,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,2,-48.118023,-15.846471
4,cvrp-2-df-33,df-2,-48.054989,-15.838145,Taguatinga,Taguatinga,180,7,-48.114898,-15.858055


In [24]:
# Definindo caminhos
shapefile_path = "distrito-federal.shp"
shxfile_path = "distrito-federal.shx"

# Lendo o arquivo shp
mapa_df = gpd.read_file(shapefile_path, vfs=f'{shxfile_path}')
mapa_df = mapa_df.loc[[0]]

# Definindo CRS como 'EPSG:4326'
if mapa_df.crs is None:
    mapa_df.crs = 'EPSG:4326'

# Convertendo o sistema de coordenadas para EPSG:32723
mapa_df = mapa_df.to_crs(epsg=32723)

# Adicionando a coluna 'Area(km)' ao DataFrame
mapa_df['Area(km)'] = mapa_df.area / 10**6

# Convertendo o GeoDataFrame para um DataFrame Pandas
mapa_df = pd.DataFrame(mapa_df)

# Exibindo o DataFrame resultante
print(mapa_df)

                                            geometry     Area(km)
0  POLYGON Z ((252785.743 8225703.192 0.000, 2527...  5784.571009


In [25]:
# Lendo o arquivo shapefile "bairros-df.shp" e armazenando os dados em um GeoDataFrame chamado 'area_adm'
area_adm = gpd.read_file("bairros-df.shp")

# Definindo o sistema de coordenadas do GeoDataFrame como 'EPSG:32723'
area_adm.crs = 'EPSG:32723'

# Transformando o sistema de coordenadas do GeoDataFrame para 'EPSG:4326'
area_adm = area_adm.to_crs(epsg=4326)

# Convertendo o GeoDataFrame para um DataFrame Pandas e o resetando para ter um índice numérico padrão
area_adm = pd.DataFrame(area_adm).reset_index()

# Incrementando todos os valores no índice em 1
area_adm["index"] = area_adm["index"] + 1

# Renomeando a coluna "index" para "id"
area_adm = area_adm.rename(columns={"index": "id"})

# Exibindo o DataFrame resultante
area_adm


,id,geometry
0,1,"POLYGON ((-47.85950 -15.68823, -47.85951 -15.6..."
1,2,"POLYGON ((-48.03822 -15.94543, -48.03714 -15.9..."
2,3,"POLYGON ((-48.20825 -15.74034, -48.20778 -15.7..."
3,4,"POLYGON ((-47.76379 -15.52734, -47.76379 -15.5..."
4,5,"POLYGON ((-47.41841 -15.50182, -47.41841 -15.5..."
5,6,"POLYGON ((-47.66188 -15.73406, -47.66217 -15.7..."
6,7,"POLYGON ((-47.95455 -15.84839, -47.95465 -15.8..."
7,8,"POLYGON ((-47.97051 -15.80634, -47.97056 -15.8..."
8,9,"POLYGON ((-47.93818 -15.80552, -47.93963 -15.8..."
9,10,"POLYGON ((-48.04743 -15.85948, -48.04742 -15.8..."


In [27]:
# Lendo o arquivo Excel "dimRA.xlsx" e armazenando os dados em um DataFrame Pandas chamado 'dRA'
dRA = pd.read_excel("dimRA.xlsx")

# Filtrando apenas as colunas desejadas no DataFrame 'dRA' (id, RA, Pop 2015, TMGCA (%), AU (ha), AT (ha), DU 2015 (hab./ha))
# e renomeando essas colunas para nomes mais curtos e informativos
dRA = dRA[["id", "RA", "Pop 2015", "TMGCA (%)", "AU (ha)", "AT (ha)", "DU 2015 (hab./ha)"]].rename(columns={"Pop 2015": "Pop", "TMGCA (%)": "Var %", "DU 2015 (hab./ha)": "DU"})

# Realizando uma operação de junção (merge) entre os DataFrames 'area_adm' e 'dRA'
# Utilizando a coluna "id" como chave de junção e mantendo apenas as linhas comuns a ambos os DataFrames (how="inner")
dimRA = pd.merge(left=area_adm, right=dRA, how="inner", on="id")

# Exibindo o DataFrame resultante da junção
dimRA


,id,geometry,RA,Pop,Var %,AU (ha),AT (ha),DU
0,1,"POLYGON ((-47.85950 -15.68823, -47.85951 -15.6...",Plano Piloto,210067,-1.49,10176.75,40989.31,20.64
1,2,"POLYGON ((-48.03822 -15.94543, -48.03714 -15.9...",Gama,134111,-0.31,2631.71,27559.42,50.96
2,3,"POLYGON ((-48.20825 -15.74034, -48.20778 -15.7...",Taguatinga,207045,-1.38,2572.11,8056.15,80.50
3,4,"POLYGON ((-47.76379 -15.52734, -47.76379 -15.5...",Brazlândia,51816,0.68,554.03,47684.84,93.52
4,5,"POLYGON ((-47.41841 -15.50182, -47.41841 -15.5...",Sobradinho,62763,-0.75,1503.93,20122.20,41.73
5,6,"POLYGON ((-47.66188 -15.73406, -47.66217 -15.7...",Planaltina,190495,1.37,2989.46,153847.95,63.72
6,7,"POLYGON ((-47.95455 -15.84839, -47.95465 -15.8...",Paranoá,44975,-1.37,487.46,78876.96,92.26
7,8,"POLYGON ((-47.97051 -15.80634, -47.97056 -15.8...",Núcleo Bandeirante,23562,-0.32,299.77,466.94,78.60
8,9,"POLYGON ((-47.93818 -15.80552, -47.93963 -15.8...",Ceilândia,479713,3.03,3691.92,23401.14,129.94
9,10,"POLYGON ((-48.04743 -15.85948, -48.04742 -15.8...",Guará,133171,5.38,1810.57,2562.92,73.55


## 5\. Visualização